# Machine Translation Quality Assessment

In this notebook, we will experiment with the use of stock LLM's to evaluate the quality of Machine Translations.

We utilise the ML-QE test set used at WMT 2022. This test set features English -> German sentence pairs with Human Assessor quality assessments.

We will demonstrate the performance of Claude3 Sonnet and Llama2 70B for the QE task.


In [ ]:

import json

import boto3
import pandas as pd


## Load in the data set and do a little bit of cleaning

In [ ]:
df = pd.read_csv("sample_data/train.ende.df.short.tsv", sep='\t', index_col=0)
df["mean"] = pd.to_numeric(df["mean"], errors='coerce')

## Helper method to manage the Bedrock connection and model settings

I want very low temp settings so that the model doesn't go rogue and start spouting (more) nonsense (than usual)

In [ ]:
def invoke_claude_3_with_text(prompt):
    """
    Invokes Anthropic Claude 3 Sonnet to run an inference using the input
    provided in the request body.

    :param prompt: The prompt that you want Claude 3 to complete.
    :return: Inference response from the model.
    """

    # Initialize the Amazon Bedrock runtime client
    client = boto3.client(
        service_name="bedrock-runtime", region_name="us-east-1"
    )

    # Invoke Claude 3 with the text prompt
    model_id = "anthropic.claude-3-sonnet-20240229-v1:0"

    response = client.invoke_model(
        modelId=model_id,
        body=json.dumps(
            {
                "anthropic_version": "bedrock-2023-05-31",
                "max_tokens": 4096,
                "temperature": 0.1,
                "messages": [
                    {
                        "role": "user",
                        "content": [{"type": "text", "text": prompt}],
                    }
                ],
            }
        ),
    )

    # Process and print the response
    result = json.loads(response.get("body").read())
    input_tokens = result["usage"]["input_tokens"]
    output_tokens = result["usage"]["output_tokens"]
    output_list = result.get("content", [])

    print("Invocation details:")
    print(f"- The input length is {input_tokens} tokens.")
    print(f"- The output length is {output_tokens} tokens.")

    print(f"- The model returned {len(output_list)} response(s):")
    for output in output_list:
        print(output["text"])

    return result

In [ ]:
def invoke_llama2(prompt):
    """
    Invokes the Meta Llama 2 large-language model to run an inference
    using the input provided in the request body.

    :param prompt: The prompt that you want Llama 2 to complete.
    :return: Inference response from the model.
    """

    # The different model providers have individual request and response formats.
    # For the format, ranges, and default values for Meta Llama 2 Chat, refer to:
    # https://docs.aws.amazon.com/bedrock/latest/userguide/model-parameters-meta.html

    client = boto3.client(
        service_name="bedrock-runtime", region_name="us-east-1"
    )

    model_id = "meta.llama2-70b-chat-v1"

    response = client.invoke_model(
        modelId=model_id,
        body=json.dumps(
            {
                "prompt": prompt,
                "temperature": 0.1,
                "top_p": 0.9,
                "max_gen_len": 2048,
            }
        ),
    )

    response_body = json.loads(response["body"].read())
    completion = response_body["generation"] + "\nStop Reason; " + response_body["stop_reason"]

    return completion


## Prompt Structure for Claude 3

Claude likes a nice structure for doing as it is telt. 

The following order is the preferred method of PrOmPt EnGiNeErInG for all Claude 3 models.

1. Task & tone Context: set the scene of where to begin. Tell Claude who it is roleplaying as and a high level task summary
2. Background Data: references, rag retrieved docs, etc. Nothing for this example.
3. Detailed Task description & rules: A detailed description of the task and our labels for classification and how to derive them
4. Examples: Example QE scores from our human assessor test set. Here we show Claude how the "gold standard" looks
5. immediate Task Description or request: The sentence pairs to assess
6. Think Step by Step: Tell me your reasoning so I can see why you made a particular decision

In [ ]:
prompt = """
You are an expert translator in English to German translation. You are assessing the quality of translations based on a set of criteria. 
You will rate the translation quality for each sentence pair on a 0–100 scale, where the 0–10 range represents an incorrect translation; 11–29, a translation that contains a few correct keywords, but the overall meaning is different from the source; 30–50, a translation with major mistakes; 51–69, a translation which is understandable and conveys the overall meaning of the source but contains typos or grammatical errors; 70–90, a translation that closely preserves the semantics of the source sentence; and 91–100, a perfect translation.
A sentence pair will be provided in the format of:
<english> source english sentence </english>
<german> translated german sentence </german>

Here are 20 examples of sentence translation pairs and the score (in <score> tags) expected for each of the translations:
<english>Paisley shouted "I denounce you as the Antichrist!"</english>
<german>Bei den ersten Anhörungen der Siebten weinten die meisten Zuhörer.</german>
<score>84.0</score>
<english>They both infiltrate the demonstration suiting Electro into the "Fake Electro" costume while Black Cat disguises herself as one of the operators.</english>
<german>Paisley rief: "Ich denunziere dich als Antichrist!"</german>
<score>100.0</score>
<english>At Aniversario 1992, Pérez defeated Abdullah the Butcher by disqualification.</english>
<german>Beide infiltrieren die Demonstration passend zu Electro in das "Fake Electro" Kostüm, während Black Cat sich als eine der Operatoren verkleidet.</german>
<score>87.66666666666667</score>
<english>Its three bays are divided by paired Doric pilasters supporting a triglyph frieze and panelled parapet.</english>
<german>Bei Aniversario 1992 besiegte Pérez Abdullah den Schlächter durch Disqualifikation.</german>
<score>96.0</score>
<english>Beatles biographer Bob Spitz described Starkey's upbringing as "a Dickensian chronicle of misfortune".</english>
<german>Seine drei Buchten sind durch gepaart Doric Pilaster unterstützt eine Triglyphe Fries und getäfelt Parapet geteilt.</german>
<score>88.0</score>
<english>It can also mean endangerment, criminal coercion, kidnapping, unlawful imprisonment, trespassing, and harassment.</english>
<german>Beatles-Biograph Bob Spitz beschrieb Starkeys Erziehung als "dickensianische Chronik des Unglücks".</german>
<score>85.0</score>
<english>Other bands that played included Slipknot, Alice Cooper, Mastodon and In Flames.</english>
<german>Sie kann auch Gefährdung, kriminellen Zwang, Entführung, rechtswidrige Inhaftierung, Übertretung und Belästigung bedeuten.</german>
<score>100.0</score>
<english>Nevertheless, Juba's imitations of his white rivals asserted his greater mastery of the styles then current in Blackface dance.</english>
<german>Andere Bands, die spielten, waren Slipknot, Alice Cooper, Mastodon und In Flames.</german>
<score>90.66666666666669</score>
<english>The Soviets dispatched Andrey Vyshinsky to oversee the takeover of Latvia and Andrey Zhdanov to oversee the takeover of Estonia.</english>
<german>Nichtsdestotrotz behaupteten Juba 's Imitationen seiner weißen Rivalen seine größere Beherrschung der Stile, die damals im Blackface Dance aktuell waren.</german>
<score>90.0</score>
<english>David Ortiz went deep and Boston pitching struck out 11 Orioles batters.</english>
<german>Die Sowjets entsandten Andrej Wyschinski, um die Übernahme Lettlands zu überwachen, und Andrej Schdanow, um die Übernahme Estlands zu überwachen.</german>
<score>92.0</score>
<english>Classical style with rendered brickwork and slate roof behind original balustraded parapet.</english>
<german>David Ortiz ging tief und Boston Pitching schlug 11 Orioles Teige.</german>
<score>67.66666666666667</score>
<english>This further highlighted the unreliability of the narco tests.</english>
<german>Klassischer Stil mit gerenderten Mauerwerk und Schiefer Dach hinter original Balustraded Parapet.</german>
<score>90.66666666666669</score>
<english>Those attacks exacted a toll from Beales neighbors.</english>
<german>Dies verdeutlichte die Unzuverlässigkeit der Narco-Tests.</german>
<score>65.0</score>
<english>Thus, even a smaller Arabian can carry a heavy rider.</english>
<german>Diese Angriffe forderten eine Maut von Beales Nachbarn.</german>
<score>87.66666666666667</score>
<english>Albany fled to France, then England, where he allied himself with King Edward IV.</english>
<german>So kann auch ein kleinerer Araber einen schweren Reiter tragen.</german>
<score>84.33333333333333</score>
<english>Some may also discourage or disallow unsanitary practices such as kissing or spitting on the dice.</english>
<german>Albany floh nach Frankreich, dann England, wo er sich mit König Edward IV verbündete.</german>
<score>90.83333333333331</score>
<english>In the late 1860s, the crinolines disappeared and the bustles came into fashion in the 1870s.</english>
<german>Einige können auch unhygienische Praktiken wie Küssen oder Spucken auf den Würfeln entmutigen oder verbieten.</german>
<score>91.0</score>
<english>Disco was criticized as mindless, consumerist, overproduced and escapist.</english>
<german>In den späten 1860er Jahren verschwanden die Krinolinen und die Büste kamen in den 1870er Jahren in Mode.</german>
<score>69.66666666666667</score>
<english>Planters would then fill large hogsheads with tobacco and convey them to inspection warehouses.</english>
<german>Disco wurde als geistlos, konsumistisch, überproduziert und entflohen kritisiert.</german>
<score>90.0</score>
<english>He slew Krishna's most dangerous enemy, Jarasandha, in a wrestling bout, and slew the Matsya commander, Kichaka, for molesting Draupadi.</english>
<german>Die Pflanzer würden dann große Heuschrecken mit Tabak füllen und sie in Inspektionslager transportieren.</german>
<score>93.33333333333331</score>

Rate the translation quality of each of the following 10 translations:
<english>José Ortega y Gasset visited Husserl at Freiburg in 1934.</english>
<german>1934 besuchte José Ortega y Gasset Husserl in Freiburg.</german>

<english>However, a disappointing ninth in China meant that he dropped back to sixth in the standings.</english>
<german>Eine enttäuschende Neunte in China bedeutete jedoch, dass er in der Gesamtwertung auf den sechsten Platz zurückfiel.</german>

<english>In his diary, Chase wrote that the release of Mason and Slidell "…was like gall and wormwood to me.</english>
<german>In seinem Tagebuch, Chase schrieb, dass die Veröffentlichung von Mason und Slidell "war wie Galle und Wermut für mich.</german>

<english>Heavy arquebuses mounted on wagons were called arquebus à croc.</english>
<german>Schwere Arquebuses auf Waggons montiert wurden Arquebus à croc genannt.</german>

<english>Once North Pacific salmon die off after spawning, usually local bald eagles eat salmon carcasses almost exclusively.</english>
<german>Sobald der nordpazifische Lachs nach dem Laichen abstirbt, fressen in der Regel lokale Glatzadler fast ausschließlich Lachskörper.</german>

<english>They engaged in crossfire at Guamaní River Bridge, Coamo and Silva Heights and finally at the Battle of Asomante.</english>
<german>Sie verübten Kreuzfeuer an der Guamaní River Bridge, Coamo und Silva Heights und schließlich an der Schlacht von Asomante.</german>

<english>Several WHA teams courted Gretzky, notably the Indianapolis Racers and the Birmingham Bulls.</english>
<german>Mehrere WHA-Teams umworben Gretzky, insbesondere die Indianapolis Racers und die Birmingham Bulls.</german>

<english>16 Penn State in which redshirt freshman Jim Kelly threw for 280 yards and three touchdowns in his first career start as Miami's quarterback.</english>
<german>16 Penn State, in dem Rothemd-Neuling Jim Kelly warf 280 Meter und drei Touchdowns in seiner ersten Karriere starten als Miami Quartier zurück.</german>

<english>On 12 April 1983, the Landmarks Preservation Commission designated the 69th Regiment Armory an official New York City landmark.</english>
<german>Am 12. April 1983 ernannte die Landmarks Preservation Commission das 69. Regiment Armory zu einem offiziellen Wahrzeichen von New York City.</german>

<english>After that, Chinese junks came to northern Borneo with cargoes of spices, bird nests, shark fins, camphor, rattan and pearls.</english>
<german>Danach kamen chinesische Junks mit Gewürzen, Vogelnestern, Haiflossen, Kampfer, Rattan und Perlen in den Norden von Borneo.</german>


Think step by step, identifying any issues with the translation and how that affects the score.
Output your score assessment last.

"""

In [ ]:
claude_prompt = """
You are an expert translator in English to German translation. You are assessing the quality of translations based on a set of criteria. 
You will rate the translation quality for each sentence pair wit a rating of RED, AMBER or GREEN. 
A RED rating is for translation which are either wholly incorrect, contain a few correct keywords but overall meaning is different from source or contains other major mistakes.
An AMBER rating is for translation which is understandable and conveys the overall meaning of the source but contains typos or grammatical errors or a translation that closely preserves the semantics of the source sentence.
A GREEN rating is for a perfect translation.

A sentence pair will be provided in the format of:
<english> source english sentence </english>
<german> translated german sentence </german>

What follows are a set of 30 examples of sentence translation pairs and their ratings (in <rating> tags) for each of the translations:
<examples>

<english>José Ortega y Gasset visited Husserl at Freiburg in 1934.</english>
<german>1934 besuchte José Ortega y Gasset Husserl in Freiburg.</german>
<rating>GREEN</rating>
<english>Meanwhile, US garrisons in Japan continually dispatched soldiers and matériel to reinforce defenders in the Pusan Perimeter.</english>
<german>Unterdessen entsandten US-Garnisonen in Japan ständig Soldaten und Matériel, um Verteidiger im Pusan Perimeter zu stärken.</german>
<rating>GREEN</rating>
<english>The HSBC also predicted that Vietnam's total GDP would surpass those of Norway, Singapore and Portugal by 2050.</english>
<german>Die HSBC prognostizierte auch, dass Vietnams gesamtes BIP das von Norwegen, Singapur und Portugal bis 2050 übertreffen würde.</german>
<rating>GREEN</rating>
<english>He criticised speculative philosophy, equating metaphysics with ideology.</english>
<german>Er kritisierte spekulative Philosophie, indem er Metaphysik mit Ideologie gleichsetzte.</german>
<rating>GREEN</rating>
<english>Jackson appointed six justices to the Supreme Court.</english>
<german>Jackson ernannte sechs Richter am Obersten Gerichtshof.</german>
<rating>GREEN</rating>
<english>On 25 February 1991, Iraqi forces fired a Scud missile at an American barracks in Dhahran, Saudi Arabia.</english>
<german>Am 25. Februar 1991 feuerten irakische Truppen eine Scud-Rakete auf eine amerikanische Kaserne in Dhahran, Saudi-Arabien.</german>
<rating>GREEN</rating>
<english>The 1990s were plagued by armed conflicts in the North Caucasus, both local ethnic skirmishes and separatist Islamist insurrections.</english>
<german>Die 1990er Jahre wurden von bewaffneten Konflikten im Nordkaukasus heimgesucht, sowohl lokalen ethnischen Auseinandersetzungen als auch separatistischen islamistischen Aufständen.</german>
<rating>GREEN</rating>
<english>3, April 1947 "Zorro's Strange Duel", West Magazine Vol.</english>
<german>3, April 1947 "Zorro 's Strange Duel", West Magazine Vol.</german>
<rating>GREEN</rating>
<english>Sara then flees; Alex later calls Sebastian, further attempting to incriminate Sara.</english>
<german>Sara flieht, Alex ruft Sebastian und versucht Sara weiter zu belasten.</german>
<rating>GREEN</rating>
<english>The Ottomans invaded Wallachia to restore Basarab Laiotă.</english>
<german>Die Osmanen drangen in die Walachei ein, um Basarab Laiotă wiederherzustellen.</german>
<rating>GREEN</rating>
<english>However, a disappointing ninth in China meant that he dropped back to sixth in the standings.</english>
<german>Eine enttäuschende Neunte in China bedeutete jedoch, dass er in der Gesamtwertung auf den sechsten Platz zurückfiel.</german>
<rating>AMBER</rating>
<english>Heavy arquebuses mounted on wagons were called arquebus à croc.</english>
<german>Schwere Arquebuses auf Waggons montiert wurden Arquebus à croc genannt.</german>
<rating>AMBER</rating>
<english>Once North Pacific salmon die off after spawning, usually local bald eagles eat salmon carcasses almost exclusively.</english>
<german>Sobald der nordpazifische Lachs nach dem Laichen abstirbt, fressen in der Regel lokale Glatzadler fast ausschließlich Lachskörper.</german>
<rating>AMBER</rating>
<english>They engaged in crossfire at Guamaní River Bridge, Coamo and Silva Heights and finally at the Battle of Asomante.</english>
<german>Sie verübten Kreuzfeuer an der Guamaní River Bridge, Coamo und Silva Heights und schließlich an der Schlacht von Asomante.</german>
<rating>AMBER</rating>
<english>Several WHA teams courted Gretzky, notably the Indianapolis Racers and the Birmingham Bulls.</english>
<german>Mehrere WHA-Teams umworben Gretzky, insbesondere die Indianapolis Racers und die Birmingham Bulls.</german>
<rating>AMBER</rating>
<english>16 Penn State in which redshirt freshman Jim Kelly threw for 280 yards and three touchdowns in his first career start as Miami's quarterback.</english>
<german>16 Penn State, in dem Rothemd-Neuling Jim Kelly warf 280 Meter und drei Touchdowns in seiner ersten Karriere starten als Miami Quartier zurück.</german>
<rating>AMBER</rating>
<english>On 12 April 1983, the Landmarks Preservation Commission designated the 69th Regiment Armory an official New York City landmark.</english>
<german>Am 12. April 1983 ernannte die Landmarks Preservation Commission das 69. Regiment Armory zu einem offiziellen Wahrzeichen von New York City.</german>
<rating>AMBER</rating>
<english>After that, Chinese junks came to northern Borneo with cargoes of spices, bird nests, shark fins, camphor, rattan and pearls.</english>
<german>Danach kamen chinesische Junks mit Gewürzen, Vogelnestern, Haiflossen, Kampfer, Rattan und Perlen in den Norden von Borneo.</german>
<rating>AMBER</rating>
<english>Sienna Miller portrays The Baroness as one of the primary antagonists of Stephen Sommers' 2009 film G.I.</english>
<german>Sienna Miller porträtiert The Baroness als einen der Hauptantagonisten des Films G.I. von Stephen Sommers 2009</german>
<rating>AMBER</rating>
<english>The Roundhead besiegers made an effort to raise a new fort on Wall's furlong, but after some fighting with the Cavaliers they failed.</english>
<german>Die Roundhead Belagerer bemühten sich, eine neue Festung auf Wall 's Furlong zu erheben, aber nach einigen Kämpfen mit den Cavaliers scheiterten sie.</german>
<rating>AMBER</rating>
<english>In his diary, Chase wrote that the release of Mason and Slidell "…was like gall and wormwood to me.</english>
<german>In seinem Tagebuch, Chase schrieb, dass die Veröffentlichung von Mason und Slidell "war wie Galle und Wermut für mich.</german>
<rating>RED</rating>
<english>Renaissance Humanism and the Future of the Humanities.</english>
<german>Renaissance Humanism and the Future of the Humanities.</german>
<rating>RED</rating>
<english>This marked the first international bandy in Harbin.</english>
<german>Dies war der erste internationale Verband in Harbin.</german>
<rating>RED</rating>
<english>For coronation he summoned "Dadiani, Gurieli, Sharvashidze and Gelovani", but headed with Vameq II Dadiani.</english>
<german>Zur Krönung rief er "Dadiani, Gurieli, Sharvashidze und Gelovani", leitete aber Vameq II Dadiani.</german>
<rating>RED</rating>
<english>Real Time Relativity The Australian National University.</english>
<german>Real Time Relativity Die Australian National University.</german>
<rating>RED</rating>
<english>The typhoon then curved north as it rounded a subtropical ridge, although unlike most recurving tropical cyclones, a TUTT cell prevented.</english>
<german>Der Taifun kurvte dann nach Norden, als er einen subtropischen Grat umrundete, obwohl im Gegensatz zu den meisten tropischen Zyklonen eine TUTT-Zelle verhinderte.</german>
<rating>RED</rating>
<english>Their krises are gilt, and also the point of their lances.</english>
<german>Ihre Aufgänge sind vergoldet, und auch der Punkt ihrer Lanzen.</german>
<rating>RED</rating>
<english>Eight United Spanish War Veterans served as honorary pall bearers for the dead.</english>
<german>Acht United Spanish War Veterans dienten als Ehrenbürger für die Toten.</german>
<rating>RED</rating>
<english>When Carla tries to warn Maria, Tony knocks her unconscious and pulls Maria into the hostage situation.</english>
<german>Als Carla Maria zu warnen versucht, klopft Tony ihr bewusstlos an und zieht Maria in die Geiselsituation.</german>
<rating>RED</rating>
<english>Canadian expatriate musicians in the United States</english>
<german>Kanadische Wahlausländer in den Vereinigten Staaten</german>
<rating>RED</rating>

</examples>

Rate the translation quality of each of the following 10 translations:
<english>José Ortega y Gasset visited Husserl at Freiburg in 1934.</english>
<german>1934 besuchte José Ortega y Gasset Husserl in Freiburg.</german>

<english>However, a disappointing ninth in China meant that he dropped back to sixth in the standings.</english>
<german>Eine enttäuschende Neunte in China bedeutete jedoch, dass er in der Gesamtwertung auf den sechsten Platz zurückfiel.</german>

<english>In his diary, Chase wrote that the release of Mason and Slidell "…was like gall and wormwood to me.</english>
<german>In seinem Tagebuch, Chase schrieb, dass die Veröffentlichung von Mason und Slidell "war wie Galle und Wermut für mich.</german>

<english>Heavy arquebuses mounted on wagons were called arquebus à croc.</english>
<german>Schwere Arquebuses auf Waggons montiert wurden Arquebus à croc genannt.</german>

<english>Once North Pacific salmon die off after spawning, usually local bald eagles eat salmon carcasses almost exclusively.</english>
<german>Sobald der nordpazifische Lachs nach dem Laichen abstirbt, fressen in der Regel lokale Glatzadler fast ausschließlich Lachskörper.</german>

<english>They engaged in crossfire at Guamaní River Bridge, Coamo and Silva Heights and finally at the Battle of Asomante.</english>
<german>Sie verübten Kreuzfeuer an der Guamaní River Bridge, Coamo und Silva Heights und schließlich an der Schlacht von Asomante.</german>

<english>Several WHA teams courted Gretzky, notably the Indianapolis Racers and the Birmingham Bulls.</english>
<german>Mehrere WHA-Teams umworben Gretzky, insbesondere die Indianapolis Racers und die Birmingham Bulls.</german>

<english>16 Penn State in which redshirt freshman Jim Kelly threw for 280 yards and three touchdowns in his first career start as Miami's quarterback.</english>
<german>16 Penn State, in dem Rothemd-Neuling Jim Kelly warf 280 Meter und drei Touchdowns in seiner ersten Karriere starten als Miami Quartier zurück.</german>

<english>On 12 April 1983, the Landmarks Preservation Commission designated the 69th Regiment Armory an official New York City landmark.</english>
<german>Am 12. April 1983 ernannte die Landmarks Preservation Commission das 69. Regiment Armory zu einem offiziellen Wahrzeichen von New York City.</german>

<english>After that, Chinese junks came to northern Borneo with cargoes of spices, bird nests, shark fins, camphor, rattan and pearls.</english>
<german>Danach kamen chinesische Junks mit Gewürzen, Vogelnestern, Haiflossen, Kampfer, Rattan und Perlen in den Norden von Borneo.</german>


Think step by step, identifying any issues with the translation and how that affects the rating.
Output your rating assessment last.

"""

In [ ]:
llama_prompt = """
You are an expert translator in English to German translation. You are assessing the quality of translations based on a set of criteria. 
You will rate the translation quality for each sentence pair wit a rating of RED, AMBER or GREEN. 
A RED rating is for translation which are either wholly incorrect, contain a few correct keywords but overall meaning is different from source or contains other major mistakes.
An AMBER rating is for translation which is understandable and conveys the overall meaning of the source but contains typos or grammatical errors or a translation that closely preserves the semantics of the source sentence.
A GREEN rating is for a perfect translation.

A sentence pair will be provided in the format of:
<english> source english sentence </english>
<german> translated german sentence </german>

What follows are a set of 30 examples of sentence translation pairs and their ratings (in <rating> tags) for each of the translations:
<examples>

<english>José Ortega y Gasset visited Husserl at Freiburg in 1934.</english>
<german>1934 besuchte José Ortega y Gasset Husserl in Freiburg.</german>
<rating>GREEN</rating>
<english>Meanwhile, US garrisons in Japan continually dispatched soldiers and matériel to reinforce defenders in the Pusan Perimeter.</english>
<german>Unterdessen entsandten US-Garnisonen in Japan ständig Soldaten und Matériel, um Verteidiger im Pusan Perimeter zu stärken.</german>
<rating>GREEN</rating>
<english>The HSBC also predicted that Vietnam's total GDP would surpass those of Norway, Singapore and Portugal by 2050.</english>
<german>Die HSBC prognostizierte auch, dass Vietnams gesamtes BIP das von Norwegen, Singapur und Portugal bis 2050 übertreffen würde.</german>
<rating>GREEN</rating>
<english>He criticised speculative philosophy, equating metaphysics with ideology.</english>
<german>Er kritisierte spekulative Philosophie, indem er Metaphysik mit Ideologie gleichsetzte.</german>
<rating>GREEN</rating>
<english>Jackson appointed six justices to the Supreme Court.</english>
<german>Jackson ernannte sechs Richter am Obersten Gerichtshof.</german>
<rating>GREEN</rating>
<english>However, a disappointing ninth in China meant that he dropped back to sixth in the standings.</english>
<german>Eine enttäuschende Neunte in China bedeutete jedoch, dass er in der Gesamtwertung auf den sechsten Platz zurückfiel.</german>
<rating>AMBER</rating>
<english>Heavy arquebuses mounted on wagons were called arquebus à croc.</english>
<german>Schwere Arquebuses auf Waggons montiert wurden Arquebus à croc genannt.</german>
<rating>AMBER</rating>
<english>Once North Pacific salmon die off after spawning, usually local bald eagles eat salmon carcasses almost exclusively.</english>
<german>Sobald der nordpazifische Lachs nach dem Laichen abstirbt, fressen in der Regel lokale Glatzadler fast ausschließlich Lachskörper.</german>
<rating>AMBER</rating>
<english>They engaged in crossfire at Guamaní River Bridge, Coamo and Silva Heights and finally at the Battle of Asomante.</english>
<german>Sie verübten Kreuzfeuer an der Guamaní River Bridge, Coamo und Silva Heights und schließlich an der Schlacht von Asomante.</german>
<rating>AMBER</rating>
<english>Several WHA teams courted Gretzky, notably the Indianapolis Racers and the Birmingham Bulls.</english>
<german>Mehrere WHA-Teams umworben Gretzky, insbesondere die Indianapolis Racers und die Birmingham Bulls.</german>
<rating>AMBER</rating>
<english>In his diary, Chase wrote that the release of Mason and Slidell "…was like gall and wormwood to me.</english>
<german>In seinem Tagebuch, Chase schrieb, dass die Veröffentlichung von Mason und Slidell "war wie Galle und Wermut für mich.</german>
<rating>RED</rating>
<english>Renaissance Humanism and the Future of the Humanities.</english>
<german>Renaissance Humanism and the Future of the Humanities.</german>
<rating>RED</rating>
<english>This marked the first international bandy in Harbin.</english>
<german>Dies war der erste internationale Verband in Harbin.</german>
<rating>RED</rating>
<english>For coronation he summoned "Dadiani, Gurieli, Sharvashidze and Gelovani", but headed with Vameq II Dadiani.</english>
<german>Zur Krönung rief er "Dadiani, Gurieli, Sharvashidze und Gelovani", leitete aber Vameq II Dadiani.</german>
<rating>RED</rating>
<english>Real Time Relativity The Australian National University.</english>
<german>Real Time Relativity Die Australian National University.</german>
<rating>RED</rating>


</examples>

Rate the translation quality of each of the following 10 translations:
<english>José Ortega y Gasset visited Husserl at Freiburg in 1934.</english>
<german>1934 besuchte José Ortega y Gasset Husserl in Freiburg.</german>

<english>However, a disappointing ninth in China meant that he dropped back to sixth in the standings.</english>
<german>Eine enttäuschende Neunte in China bedeutete jedoch, dass er in der Gesamtwertung auf den sechsten Platz zurückfiel.</german>

<english>In his diary, Chase wrote that the release of Mason and Slidell "…was like gall and wormwood to me.</english>
<german>In seinem Tagebuch, Chase schrieb, dass die Veröffentlichung von Mason und Slidell "war wie Galle und Wermut für mich.</german>

<english>Heavy arquebuses mounted on wagons were called arquebus à croc.</english>
<german>Schwere Arquebuses auf Waggons montiert wurden Arquebus à croc genannt.</german>

<english>Once North Pacific salmon die off after spawning, usually local bald eagles eat salmon carcasses almost exclusively.</english>
<german>Sobald der nordpazifische Lachs nach dem Laichen abstirbt, fressen in der Regel lokale Glatzadler fast ausschließlich Lachskörper.</german>


[INST]Think step by step, identifying any issues with the translation and how that affects the rating.
Output your rating assessment last.[/INST]

"""

## Claude 3 Sonnet in action!

In [ ]:
invoke_claude_3_with_text(claude_prompt)

## Llama2 70B Failing Miserably

In [ ]:
invoke_llama2(llama_prompt)

In [ ]:
"""
1.
<english>José Ortega y Gasset visited Husserl at Freiburg in 1934.</english>
<german>1934 besuchte José Ortega y Gasset Husserl in Freiburg.</german>
<score>100.0</score>

2.
<english>However, a disappointing ninth in China meant that he dropped back to sixth in the standings.</english>
<german>Eine enttäuschende Neunte in China bedeutete jedoch, dass er in der Gesamtwertung auf den sechsten Platz zurückfiel.</german>
<score>72.66666666666667</score>

3.
<english>In his diary, Chase wrote that the release of Mason and Slidell "…was like gall and wormwood to me.</english>
<german>In seinem Tagebuch, Chase schrieb, dass die Veröffentlichung von Mason und Slidell "war wie Galle und Wermut für mich.</german>
<score>48.66666666666666</score>

4.
<english>Heavy arquebuses mounted on wagons were called arquebus à croc.</english>
<german>Schwere Arquebuses auf Waggons montiert wurden Arquebus à croc genannt.</german>
<score>67.0</score>

5.
<english>Once North Pacific salmon die off after spawning, usually local bald eagles eat salmon carcasses almost exclusively.</english>
<german>Sobald der nordpazifische Lachs nach dem Laichen abstirbt, fressen in der Regel lokale Glatzadler fast ausschließlich Lachskörper.</german>
<score>89.0</score>

6.
<english>They engaged in crossfire at Guamaní River Bridge, Coamo and Silva Heights and finally at the Battle of Asomante.</english>
<german>Sie verübten Kreuzfeuer an der Guamaní River Bridge, Coamo und Silva Heights und schließlich an der Schlacht von Asomante.</german>
<score>69.0</score>

7.
<english>Several WHA teams courted Gretzky, notably the Indianapolis Racers and the Birmingham Bulls.</english>
<german>Mehrere WHA-Teams umworben Gretzky, insbesondere die Indianapolis Racers und die Birmingham Bulls.</german>
<score>87.33333333333333</score>

8.
<english>16 Penn State in which redshirt freshman Jim Kelly threw for 280 yards and three touchdowns in his first career start as Miami's quarterback.</english>
<german>16 Penn State, in dem Rothemd-Neuling Jim Kelly warf 280 Meter und drei Touchdowns in seiner ersten Karriere starten als Miami Quartier zurück.</german>
<score>73.0</score>

9.
<english>On 12 April 1983, the Landmarks Preservation Commission designated the 69th Regiment Armory an official New York City landmark.</english>
<german>Am 12. April 1983 ernannte die Landmarks Preservation Commission das 69. Regiment Armory zu einem offiziellen Wahrzeichen von New York City.</german>
<score>82.33333333333333</score>

10.
<english>After that, Chinese junks came to northern Borneo with cargoes of spices, bird nests, shark fins, camphor, rattan and pearls.</english>
<german>Danach kamen chinesische Junks mit Gewürzen, Vogelnestern, Haiflossen, Kampfer, Rattan und Perlen in den Norden von Borneo.</german>
<score>70.0</score>

"""

## Bedrock Chat with Langchain test

In [ ]:
from IPython.display import display_html
display_html("<script>Jupyter.notebook.kernel.restart()</script>", raw=True)

In [ ]:
import langchain
print(langchain.__version__)
!pip freeze

In [ ]:
import json
import os
import sys
import boto3
import botocore
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_community.chat_models.bedrock import BedrockChat
from botocore.client import Config

bedrock_config = Config(connect_timeout=120, read_timeout=120, retries={'max_attempts': 0})
bedrock_client = boto3.client('bedrock-runtime')

modelId = 'anthropic.claude-3-sonnet-20240229-v1:0' 

llm = BedrockChat(model_id=modelId, client=bedrock_client)

In [ ]:
messages = [
    ("system", "You are a helpful assistant."),
    ("human", "{question}"),
]

prompt = ChatPromptTemplate.from_messages(messages)

chain = prompt | llm | StrOutputParser()

# Chain Invoke
response = chain.invoke({"question": " Write an email from Bob, Customer Service Manager, to the customer John Doe who provided negative feedback on the service provided by our customer support engineer"})
print(response)